In [1]:
import xarray as xr
import numpy as np

import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling

import os, glob
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from temporal_curve import temporal_curve_24hr
from temporal_curve import future_ddfs

In [2]:
# For the given ARI: 
# Step-1 Create future regional DDFs
#        fut_ddfs_region = future_ddfs(dgmt, cf_region, his_ddfs_region)
# Step-2 Create future regional temporal curve of 24hr, and convert to pdfs 
#        fut_crrs_region = temporal_curve_24hr(fut_ddfs_region)
#        fut_pdfs_region
# -----------------------------------------------------------
# Step-1 and Step-2 are completed by 03_Cal_future_regional temproal_curve
# -----------------------------------------------------------
# Step-3 Create future P24 for the given [ssp, ens_ptl, year], and save as geotiffs
#        P24_fut
# Step-4 Create future timeseries for each point by 
#        future_timeseries = P24_fut * fut_pdfs_region

In [3]:
ARI = 100
ssp = 'SSP5-8.5'  # #'SSP2-4.5' #'SSP5-8.5'  # 'SSP1-1.9', 'SSP2-4.5'
ens_ptl  = 50  # 50% ensemble percentile, 5, 95, 99
# years = [2023, 2025, 2030, 2035, 2036, 2037, 2040, 2045, 2050, 2100]   # RBNZ
years = [2030, 2040, 2050]  # Vector


In [4]:
ssp_short = ssp.replace('-', '').replace('.', '')
ssp_short

'SSP585'

In [5]:
# Region polygon shapefile with buffer
                            
gdf_Pilbara = gpd.read_file('/home/lusun/LuSUN/Data/riotinto_mash/catchment_boundary/watershed_WGS84_without_offshore_islands_filledholes_dissolved.shp')
gdf_Pilbara

,id,geometry
0,1,"POLYGON ((118.74750 -20.29250, 118.74750 -20.3..."


In [6]:
# Read historical regional ddfs

output_path = '/home/lusun/LuSUN/output/'

prep_dist = output_path + 'baseline_regional_ddfs_ARI' + str(ARI) + '_byPilbaraClimate.csv'
his_ddfs = pd.read_csv(prep_dist, sep=',',index_col=0)
his_ddfs

,Pilbara
mins,
1.2,5.509012
1.8,8.790863
3.0,12.525744
4.2,16.124094
4.8,19.470707
10.2,32.566658
15.0,41.523441
19.8,48.164856
25.2,53.415867


In [7]:
# Read historical regional ddfs

output_path = '/home/lusun/LuSUN/output/'

prep_dist = output_path + 'baseline_regional_pdfs_ARI' + str(ARI) + '_byPilbaraClimate.csv'
his_pdfs = pd.read_csv(prep_dist, sep=',',index_col=0)
his_pdfs

,Pilbara
hours,
0.0,0.0000
0.1,0.0009
0.2,0.0010
0.3,0.0009
0.4,0.0010
...,...
23.6,0.0011
23.7,0.0010
23.8,0.0009


In [8]:
# Read  historical regional DDF 
from pathlib import Path

data_dir=  Path("/home/lusun/LuSUN/Data/riotinto_mash")
chunks = {'time':-1}
precip_files = list(sorted(data_dir.joinpath("all_durations").glob("*Precipitation*.nc4")))
dataset  = xr.open_mfdataset(precip_files, 
                            # engine='h5netcdf',
                            #engine='netcdf4',
                            # preprocess=preprocess, 
                            chunks=chunks)#.prec.sel(time=slice("1991","2020")) #"1991", '2020'



variable_names = list(dataset.variables)
variable_names
for var_name in ['ari', 'hrs']:
    var_values = dataset[var_name].values
    print(f"Variable: {var_name}")
    print(f"Values: {var_values}")
    print()

base_data = dataset['Extreme_Precipitation'].sel(ari=ARI, hrs=24.0).drop(["ari"]).rename({"lat": "latitude", "lon":"longitude"})
base_data .rio.write_crs("epsg:32750", inplace=True)
base_data 

Variable: ari
Values: [   2    3    5   10   20   50  100  200  500 1000 2000]

Variable: hrs
Values: [2.00e-02 3.00e-02 5.00e-02 7.00e-02 8.00e-02 1.70e-01 2.50e-01 3.30e-01
 4.20e-01 5.00e-01 7.50e-01 1.00e+00 1.50e+00 2.00e+00 3.00e+00 4.50e+00
 6.00e+00 9.00e+00 1.20e+01 1.80e+01 2.40e+01 3.00e+01 3.60e+01 4.80e+01
 7.20e+01 9.60e+01 1.20e+02 1.44e+02 1.68e+02]



<xarray.DataArray 'Extreme_Precipitation' (latitude: 1361, longitude: 1761)>
dask.array<getitem, shape=(1361, 1761), dtype=float32, chunksize=(1361, 1761), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 -9.787 -9.812 -9.837 ... -43.76 -43.79
  * longitude    (longitude) float64 110.4 110.4 110.4 ... 154.3 154.4 154.4
    hrs          float64 24.0
    spatial_ref  int64 0
Attributes:
    units:       mm
    regenerate:  http://www.climsystems.com J.Yu 2023-07-03
    source:      Australia Govemment Bureau of Meteorology Design Rainfall Da...

In [9]:
hrs = dataset['hrs'].values

In [10]:
# write nc data as a geotiff
def save_xr_as_tiff(xr_data, output_path):
    # Specify the spatial information
    lon = xr_data['longitude']
    lat = xr_data['latitude']
    lon_min, lon_max = lon.min().values, lon.max().values
    lat_min, lat_max = lat.min().values, lat.max().values
    resolution = abs(lon[1] - lon[0]).values  # Assuming equal spacing

    # Create the transform for the GeoTIFF
    transform = from_origin(lon_min, lat_max, resolution, resolution)

    # Convert the DataArray to a numpy array
    array = xr_data.values

    # Write the array as a GeoTIFF
    with rasterio.open(output_path, 'w', driver='GTiff', height=array.shape[0],
                       width=array.shape[1], count=1, dtype=array.dtype,
                       crs='epsg:32750', transform=transform) as dst:
        dst.write(array, 1)
        

In [172]:
# # write his DDFs NZ as tifs
# out_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI)
# if not os.path.exists(out_path):
#     os.makedirs(out_path)
        
# for hr in hrs:
#     data_file_full = r"E:\NZ_ExtremePrep\NewZealand_Extreme_Precipitation_baseline_NIWA.nc4"
#     da_ddfs_100a_full = xr.open_dataset(data_file_full)['Extreme_Precipitation'].sel(
#                         ari=ARI, hrs=hr).drop(["ari"]).rename({"lat": "latitude", "lon":"longitude"})

#     out_file = out_path + '\\DDFs_' + str(hr) + '_NZ.tif'
#     save_xr_as_tiff(da_ddfs_100a_full, out_file)

In [12]:
# read AR6 pattern
# ens_ptl  = 50  # 50% ensemble percentile
gev_file = "data/AUS_Pilbara_Extreme_Precipitation_Pattern_AR6.nc"

# Read all CN regions
da_cf = xr.open_dataset(gev_file).pattern.sel(ari=ARI, 
                                              pth=ens_ptl, 
                                              method='nearest'
                                             ).drop(['ari', 'pth'])  
# to pandas.DataFrame
df_cf = da_cf.to_dataframe().rename({"pattern":"cf"}, axis=1)
df_cf = df_cf.rename(index=lambda x: x * 60, level='hrs')
df_cf.index.names = ['CZ', 'min']

# df_cf.head()
df_cf

cf
CZ      min            
Pilbara 180    7.835590
        360    7.567535
        720    7.299551
        1440   7.031639
        2880   6.763798
        4320   6.607154
        5760   6.496028
        7200   6.409840
        8640   6.339426
        14400  6.142164

In [13]:
dataset = xr.open_dataset(gev_file)
variable_names = list(dataset.variables)
variable_names
for var_name in ['ari', 'hrs', 'pth']:
    var_values = dataset[var_name].values
    print(f"Variable: {var_name}")
    print(f"Values: {var_values}")
    print()

Variable: ari
Values: [  2   3   5  10  15  25  50 100 150 200 300 500]

Variable: hrs
Values: [  3   6  12  24  48  72  96 120 144 240]

Variable: pth
Values: [ 1.   5.  15.9 25.  50.  75.  84.1 95.  99. ]



In [14]:
df_cf.loc[('Pilbara', 1440), 'cf']

7.031639

In [15]:
# Read AR6 GMT, mid
ptn_file = "data/IPCC_AR6_GMT_curves.csv"

df_dgmt = pd.read_csv(ptn_file, index_col=0)
# df_dgmt = df_dgmt.iloc[5::5, 0:len(df_dgmt.columns):3]
df_dgmt = df_dgmt.iloc[15:, 0:len(df_dgmt.columns):3]
df_dgmt.columns = [col_name.split(":")[0] for col_name in df_dgmt.columns]

df_dgmt

,SSP1-1.9,SSP1-2.6,SSP2-4.5,SSP3-7.0,SSP5-8.5
Year,,,,,
2020,0.52,0.47,0.40,0.40,0.43
2021,0.54,0.50,0.43,0.42,0.47
2022,0.56,0.52,0.46,0.45,0.50
2023,0.58,0.55,0.49,0.48,0.54
2024,0.60,0.57,0.52,0.51,0.57
...,...,...,...,...,...
2096,0.70,1.21,2.02,3.38,4.38
2097,0.70,1.21,2.02,3.43,4.45
2098,0.70,1.22,2.02,3.48,4.51


In [16]:
df_cf.loc[('Pilbara', 1440), 'cf'] * df_dgmt.loc[2100, 'SSP2-4.5']

14.20391098022461

In [17]:
df_dgmt_sub = df_dgmt.loc[years, ssp]
# df_dgmt_sub
for year, dgmt in df_dgmt.loc[years, ssp].iteritems():
    print(year, dgmt)

2030 0.8
2040 1.22
2050 1.7


/tmp/ipykernel_4729/350848728.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for year, dgmt in df_dgmt.loc[years, ssp].iteritems():


In [18]:
import Jpkg as J
P24_his= J.fshape2xr(base_data.rename({'latitude':'lat', 'longitude':'lon'}), gdf_Pilbara)#.rename({'latitude':'lat', 'longitude':'lon'})
P24_his

<xarray.DataArray 'Extreme_Precipitation' (lat: 181, lon: 295)>
dask.array<getitem, shape=(181, 295), dtype=float32, chunksize=(181, 295), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -20.31 -20.34 -20.36 -20.39 ... -24.76 -24.79 -24.81
  * lon      (lon) float64 113.7 113.7 113.7 113.7 ... 120.9 121.0 121.0 121.0
    hrs      float64 24.0
Attributes:
    units:       mm
    regenerate:  http://www.climsystems.com J.Yu 2023-07-03
    source:      Australia Govemment Bureau of Meteorology Design Rainfall Da...

In [19]:
region = 'Pilbara' 
df_cf.loc[(region, 1440), 'cf']
# df_cf

7.031639

In [20]:
# Get future P24 full for all ssps, ens_ptl and all years        
        
regions = ['Pilbara']
region = 'Pilbara' 

ptn_list = [df_cf.loc[(region, 1440), 'cf']]
P24_his_list = []
P24_his_list.append(P24_his)

for year, dgmt in df_dgmt.loc[years, ssp].iteritems():
# for dgmt in [4.64]:
    # year=2100
    print(year)
    merged_path = '/home/lusun/LuSUN/output/' + ssp_short + '_' + str(ens_ptl) + 'th/ARI' + str(ARI)  
    merged_tif = merged_path + '/' + 'P24_ARI' + str(ARI) + '_' + ssp_short + '_' + str(ens_ptl) + 'th_' + str(year) + '.tif'
 #   if not os.path.exists(merged_tif):
        
        # cal future by region
    print(ptn_list)
    out_tif_list = []
    for ptn in ptn_list: 
        print(ptn)
        ind = ptn_list.index(ptn)
        region = regions[ind] 
        P24_his = P24_his_list[ind]
            # print(P24_his.max())

        P24_fut = P24_his * (1.0 + dgmt * ptn / 100.0)
        print(P24_fut.max())
        spatial_resolution = 0.025
        # crs = "EPSG:4326"
        crs = "EPSG:32750"
        # His
        out_path = '/home/lusun/LuSUN/output/'
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        out_tif = out_path + '/P24_ARI' + str(ARI) + '_' + region + '.tif'
        with rasterio.open(out_tif, 
                    'w',
                    driver = 'GTiff',
                    height = P24_his.shape[0],
                    width = P24_his.shape[1],
                    count = 1,
                    dtype = str(P24_his.dtype),
                    crs = crs,
                    transform = from_origin(P24_his.coords['lon'][0],P24_his.coords['lat'][0], spatial_resolution, spatial_resolution) 
                    ) as dst:
            dst.write(P24_his, 1)
        dst.close()

            # Future
        out_path = '/home/lusun/LuSUN/output/'
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        out_tif_fut = out_path + '/P24_ARI' + str(ARI) + '_' + region + '_' + str(year) + '.tif'
        with rasterio.open(out_tif_fut,  'w',
                    driver = 'GTiff',
                    height = P24_fut.shape[0],
                    width = P24_fut.shape[1],
                    count = 1,
                    dtype = str(P24_fut.dtype),
                    crs = crs,
                    transform = from_origin(P24_fut.coords['lon'][0],P24_fut.coords['lat'][0], spatial_resolution, spatial_resolution) 
                    ) as dst:
            dst.write(P24_fut, 1)
                
        dst.close()
            

        out_tif_list.append(out_tif_fut)

            # Merge regions to NZ
        src0_files = []
        for file in out_tif_list:
            src0 = rasterio.open(file)
            src0_files.append(src0)

            # merge the rasters
        merged, out_trans = merge(src0_files)

            # create the output raster file
        out_meta = src0.meta.copy()
        out_meta.update({"driver": "GTiff",
                            "height": merged.shape[1],
                            "width": merged.shape[2],
                            "transform": out_trans,
                            "nodata": -999.0})

        merged_path = '/home/lusun/LuSUN/output/' + ssp_short + '_' + str(ens_ptl) + 'th/ARI' + str(ARI) 
        if not os.path.exists(merged_path):
            os.makedirs(merged_path)
        merged_tif = merged_path + '/' + 'P24_ARI' + str(ARI) + '_' + ssp_short + '_' + str(ens_ptl) + 'th_' + str(year) + '.tif'
        with rasterio.open(merged_tif, "w", **out_meta) as dst:
            dst.write(merged)

        for src0 in src0_files:
            src0.close()
            
print('done')      
        
# # Delete tmp files
# folder_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\P24\tmp'  # Replace with the actual path to your folder
# file_list = os.listdir(folder_path)
# for file_name in file_list:
#     file_path = os.path.join(folder_path, file_name)
#     if os.path.isfile(file_path):  # Check if the path points to a file (not a directory)
#         os.remove(file_path)
    

2030
[7.031639]
7.031639
<xarray.DataArray 'Extreme_Precipitation' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    hrs      float64 24.0
2040
[7.031639]
7.031639
<xarray.DataArray 'Extreme_Precipitation' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    hrs      float64 24.0
2050
[7.031639]
7.031639
<xarray.DataArray 'Extreme_Precipitation' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    hrs      float64 24.0
done


In [182]:
# P24_fut
for ptn in ptn_list: 
    print(ptn)

29.327461


In [183]:
# Read future regional pdf curve

fut_pdfs_full_file = '/home/lusun/LuSUN/output/future_regional_pdfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byPilbaraClimate.csv'
fut_pdfs_df = pd.read_csv(fut_pdfs_full_file, sep=',', index_col=[0, 1, 2])
fut_pdfs_df


,,,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
hours,CZ,SSP,,,,,,,,,,,,,,,,,,,,,
0.0,Pilbara,SSP1-1.9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.1,Pilbara,SSP1-1.9,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,...,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009
0.2,Pilbara,SSP1-1.9,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
0.3,Pilbara,SSP1-1.9,0.0009,0.0010,0.0010,0.0010,0.0009,0.0009,0.0009,0.0009,0.0009,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
0.4,Pilbara,SSP1-1.9,0.0010,0.0010,0.0010,0.0010,0.0011,0.0011,0.0011,0.0011,0.0011,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23.6,Pilbara,SSP5-8.5,0.0010,0.0011,0.0011,0.0011,0.0011,0.0011,0.0010,0.0010,0.0010,0.0010,...,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011
23.7,Pilbara,SSP5-8.5,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0011,0.0011,0.0010,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
23.8,Pilbara,SSP5-8.5,0.0010,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0010,0.0010,...,0.0010,0.0011,0.0011,0.0010,0.0011,0.0010,0.0011,0.0011,0.0010,0.0010


In [184]:
# Check if there are negative in fut_pdfs
negative_values = fut_pdfs_df.lt(0)

# This will create a boolean DataFrame where 'True' indicates negative values

# If you want to check if there are any negative values in the entire DataFrame, you can use the 'any' method
if negative_values.any().any():
    print("There are negative values in the DataFrame.")
else:
    print("There are no negative values in the DataFrame.")

There are no negative values in the DataFrame.


In [185]:
fut_pdfs_df.loc[(slice(None), 'Pilbara', ssp)]

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
hours,,,,,,,,,,,,,,,,,,,,,
0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.1,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,...,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009
0.2,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
0.3,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0010,0.0009,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
0.4,0.0010,0.0010,0.0010,0.0010,0.0011,0.0011,0.0011,0.0011,0.0010,0.0011,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23.6,0.0011,0.0011,0.0011,0.0011,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,...,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011
23.7,0.0010,0.0010,0.0010,0.0010,0.0011,0.0011,0.0011,0.0011,0.0010,0.0011,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
23.8,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0010,0.0009,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010


In [186]:
fut_pdfs_df.loc[(slice(None), 'Pilbara', ssp), '2100']

hours  CZ       SSP     
0.0    Pilbara  SSP1-2.6    0.0000
0.1    Pilbara  SSP1-2.6    0.0009
0.2    Pilbara  SSP1-2.6    0.0010
0.3    Pilbara  SSP1-2.6    0.0010
0.4    Pilbara  SSP1-2.6    0.0010
                             ...  
23.6   Pilbara  SSP1-2.6    0.0011
23.7   Pilbara  SSP1-2.6    0.0010
23.8   Pilbara  SSP1-2.6    0.0010
23.9   Pilbara  SSP1-2.6    0.0010
24.0   Pilbara  SSP1-2.6    0.0009
Name: 2100, Length: 241, dtype: float64

In [187]:
times = fut_pdfs_df.index.get_level_values(0).unique().values
ntimes = len(times)
times

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9,
       11. , 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. ,
       12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1,
       13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2,
       14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15

In [188]:
# Cal rainfall time series for each grid
def Generate_rainfall_timeseries_24hr_region_fut(P24, pdf):
    P = P24 * pdf
    Px = P.cumsum()
    Px[-1] = P24
    P[-1] = P24 - Px[-2]
    return P

In [189]:
def reproject_ras(in_ras, out_ras): 

    # Open the input raster file in read mode
    with rasterio.open(in_ras) as src:

        # Define the output coordinate reference system (CRS) as NZTM
        dst_crs = 'EPSG:2193'

        # Calculate the affine transform and output dimensions for the destination raster
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)

        # Create a dictionary of output raster parameters
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Open the output raster file in write mode and perform the reprojection
        with rasterio.open(out_ras, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
                

In [190]:
ssp_short[-3:]

'126'

In [191]:
# Cal future rainfall timeseries

for year in years:
# for year in [2100]:
    
    print(year)
    
    # NZTM_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\NZTM\\ARI' + str(ARI) + '\\' + ssp_short + '_' + str(ens_ptl) + 'th_' + str(year)
    # NZTM_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\NZTM\\ARI' + str(ARI) + '\\' + ssp_short + '\\' + str(ens_ptl) + 'th\\' + str(year)
    # NZTM_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\NZTM\\ARI' + str(ARI)
    NZTM_path = '/home/lusun/LuSUN/output/future_rainfall_timeseries/ARI' + str(ARI) + '/ssp' + ssp_short[-3:] + '_' + str(ens_ptl) + 'th'
    target_file = NZTM_path + '/' + 'nz_ssp' + ssp_short[-3:] +'_'+ str(ens_ptl) + 'pctile_' + str(year) + '_' + str(ARI) + 'ari_t0.0_precip.tif'
     
    if not os.path.exists(target_file):

        # P24_file = '/home/lusun/LuSUN/output/' + ssp_short + '_' + str(ens_ptl) + 'th\\ARI' + str(ARI) + '\\' + 'P24_ARI' + str(ARI) + '_' + ssp_short + '_' + str(ens_ptl) + 'th_' + str(year) + '.tif'
        # with rasterio.open(P24_file) as src:
        #     # P24_ras = src.read(1)  # Read the first band of the raster
        #     meta = src.meta.copy()  

        # regions = []
        # iterate by region
        # for index, row in gdf_nz_bf.iterrows():

        #     region = row[1]
        #     regions.append(region)

        #     geometry = row.geometry
        #     clipped_raster, clipped_transform = mask(dataset=rasterio.open(P24_file), shapes=[geometry], crop=True)
        #     meta.update({'height': clipped_raster.shape[1], 'width': clipped_raster.shape[2], 'transform': clipped_transform})
        #     arr = clipped_raster[0]

            fut_pdfs_region = fut_pdfs_df.loc[(slice(None), region, ssp), str(year)].values

            nrows, ncols = P24_fut.shape
            result_arr = np.zeros((nrows, ncols, ntimes))
            for i in range(nrows):
                for j in range(ncols):
                    if P24_fut[i, j] != 0.0 and P24_fut[i, j] != -999.0:
#                         print(np.shape(P24_fut))
#                         print(np.shape(fut_pdfs_region))
#                        print(P24_fut[i,j])
#                         print(fut_pdfs_region)
                        result = Generate_rainfall_timeseries_24hr_region_fut(P24_fut[i, j].values, fut_pdfs_region)
                        result_arr[i, j, :] = result
                    else:
                        result_arr[i, j, :] = -999.0


            # save time series as tif
            for ind_time in range(len(times)):

                time = times[ind_time]
                data = result_arr[:, :, ind_time]
                print(data)

                out_path = '/home/lusun/LuSUN/output/future_rainfall_timeseries/'
                if not os.path.exists(out_path):
                    os.makedirs(out_path)
                # out_tif = out_path + '/Pt_' + region + '_' + str(time) + '.tif'
                print(ssp_short[-3:])
                print(str(ens_ptl))
                print(str(year))
                print(str(time))
                print(str(ARI))
                out_tif = out_path + '/'+ ssp_short[-3:] +'_'+ str(ens_ptl) + 'pctile_' + str(year) + '_'  + str(time)+ '_' + str(ARI) + 'ari_precip.tif'
                with rasterio.open(out_tif, 'w',
                       driver = 'GTiff',
                       height = P24_fut.shape[0],
                       width = P24_fut.shape[1],
                       count = 1,
                       dtype = str(P24_fut.dtype),
                       crs = crs,
                       transform = from_origin(P24_fut.coords['lon'][0],P24_fut.coords['lat'][0], spatial_resolution, spatial_resolution) 
                       ) as dst:
                    dst.write(data, 1)
                dst.close()

        # src.close()  # close P24 raster


        # Merge regions to NZ
        # print('merge')
        # for time in times:
        #     tifs_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\tmp'
        #     in_tifs = []
        #     for region in regions:
        #         in_tif = tifs_path + '\\Pt_' + region + '_' + str(time) + '.tif'
        #         in_tifs.append(in_tif)

        #     src_files = []
        #     for file in in_tifs:
        #         src0 = rasterio.open(file)
        #         src_files.append(src0)

        #     # merge the rasters
        #     mosaic, out_trans = merge(src_files, nodata=-999.0)

        #     # create the output raster file
        #     out_meta = src0.meta.copy()
        #     out_meta.update({"driver": "GTiff",
        #                      "height": mosaic.shape[1],
        #                      "width": mosaic.shape[2],
        #                      "transform": out_trans,
        #                      "nodata": -999.0})


        #     merge_path =  r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\latlon'
        #     if not os.path.exists(merge_path):
        #         os.makedirs(merge_path)
        #     merge_tif =  merge_path + '\\' + 'Pt_' + str(time) + '.tif'
        #     # merge_tif =  merge_path + '\\' + 'Pt_'  + str(year) + '_' + str(time) + '.tif' # temporary use for checking
        #     with rasterio.open(merge_tif, "w", **out_meta) as dest:
        #         dest.write(mosaic)
        #     dest.close()

        #     for src0 in src_files:
        #         src0.close()



        # check if sum(Pt) == P24
        # print('Cal SumPt')
        # sum_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\latlon\sum\ARI' + str(ARI) 
        # if not os.path.exists(sum_path):
        #     os.makedirs(sum_path)
        # sum_tif = sum_path + '\\Psum_ARI' + str(ARI) + '_NZ_' + ssp_short + '_' + str(ens_ptl) + 'th_' + str(year) + '.tif'
        # if not os.path.exists(sum_tif):
        #     raster_files = []
        #     for time in times:
        #         raster_file =  r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\latlon\Pt_' + str(time) + '.tif'
        #         raster_files.append(raster_file)
        #     with rasterio.open(raster_files[0]) as src1:
        #         sum_array = src1.read(1, masked=True)
        #         for filename in raster_files[1:]:
        #             with rasterio.open(filename) as src2:
        #                 data = src2.read(1, masked=True)
        #                 sum_array += data
        #             src2.close()
        #     src1.close()

        #     with rasterio.open(sum_tif, 'w', **src1.meta) as dst:
        #         dst.write(sum_array.filled(np.nan), 1)
        #     dst.close()



        # project to NZTM
        # print('proj')
        # if not os.path.exists(NZTM_path):
        #     os.makedirs(NZTM_path)
        # for time in times:
        #     ras_WGS84 = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\latlon\Pt_' + str(time) + '.tif'
        #     # ras_NZTM = NZTM_path + '\\' + 'Pt_ARI' + str(ARI) + '_NZ_' + ssp_short + '_' + str(ens_ptl) + 'th_' + str(year) + '_' + str(time) + '.tif'
        #     ras_NZTM = NZTM_path + '\\' + 'nz_ssp' + ssp_short[-3:] +'_'+ str(ens_ptl) + 'pctile_' + str(year) + '_' + str(ARI) + 'ari_t' + str(time) + '_precip.tif'
        #     reproject_ras(ras_WGS84, ras_NZTM)

    

print('finished!')

2030
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
126
95
2030
0.0
100
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
126
95
2030
0.1
100
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
126
95
2030
0.2
100
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
126
95
2030
0.3
100
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... 

In [145]:
# def get_point_value(file, lon, lat): 
#     dat = rasterio.open(file)
#     z = dat.read()[0]
#     idx = dat.index(lon, lat, precision=1E-6)    
#     # return dat.xy(*idx), z[idx]
#     return z[idx]
P24_fut

<xarray.DataArray 'Extreme_Precipitation' (lat: 181, lon: 295)>
dask.array<mul, shape=(181, 295), dtype=float32, chunksize=(181, 295), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -20.31 -20.34 -20.36 -20.39 ... -24.76 -24.79 -24.81
  * lon      (lon) float64 113.7 113.7 113.7 113.7 ... 120.9 121.0 121.0 121.0
    hrs      float64 24.0

In [146]:
# lat = -37.156
# lon = 174.917

# # Check P24
# values = []
# diffs = []

# # baseline - P24
# base_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\baseline\P24\P24_ARI100_NZ_full.tif'
# base_value = get_point_value(base_file, lon, lat)
# values.append(base_value)
# diffs.append(0)

# # future - P24
# for year in years:
#     file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\P24\SSP245_50th\ARI100\P24_ARI100_SSP245_50th_' + str(year) + '.tif'
#     value = get_point_value(file, lon, lat)
#     values.append(value)
#     diffs.append((value - base_value) / base_value * 100.0)


# years_full = years.copy()
# years_full.insert(0, 2022)
# P24_df = pd.DataFrame({'year': years_full, 'P24': values, 'diff': diffs})
# P24_df = P24_df.set_index('year')
# P24_df


In [147]:
# df_dgmt.loc[years, ssp] * df_cf.loc[('NNI', 1440), 'cf']  

In [112]:
# ssp, ens_ptl

In [113]:
# # Check timeseries 
# # the timeseries read
# values = []
# for time in times:
#     file = r'E:\Tuflow\Cal_rainfall_timeseries\output\byNZClimate\future\timeseries\latlon\Pt_2050_' + str(time) + '.tif'
#     value = get_point_value(file, lon, lat)
#     values.append(value)
# ts_df = pd.DataFrame({'hours': times, 'Pt': values})
# ts_df = ts_df.set_index('hours')

# # the timeseries calculated
# year = 2050
# region = 'NNI'
# pdf_df = fut_pdfs_df.loc[(slice(None), region, ssp), str(year)]
# Pt_df = pd.DataFrame(P24_df.loc[year, 'P24'] * pdf_df)
# Pt_df_index1 = [index[0] for index in Pt_df.index]

# # plt.plot(ts_df.index, ts_df['Pt'], label='Point')
# # plt.plot(Pt_df_index1, Pt_df[str(year)], label='NNI')


# fig, ax = plt.subplots()
# ax.plot(ts_df.index, ts_df['Pt'], linestyle='-', linewidth=2, color='b', label='Read')
# ax.plot(Pt_df_index1, Pt_df[str(year)], linestyle=':', linewidth=2, color='r', label='Cal')


# ax.legend()

# plt.show()

In [114]:
# Pt_df.cumsum(), ts_df.cumsum()

# Old Code below!!!


In [905]:
# regions = []

# for index, row in gdf_nz_bf.iterrows():
    
#     region = row[1]
#     regions.append(region)
    
#     geometry = row.geometry
    
#     for hr in hrs:
#         his_ddf_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI) + '\\Pt_' + str(hr) + '_NZ.tif'
#         with rasterio.open(his_ddf_file) as src:
#             meta = src.meta.copy()
#         clipped_raster, clipped_transform = mask(dataset=rasterio.open(his_ddf_file), shapes=[geometry], crop=True)
#         # Update the metadata with the clipped raster dimensions and transformation
#         meta.update({'height': clipped_raster.shape[1], 'width': clipped_raster.shape[2], 'transform': clipped_transform})
#         data_array = xr.DataArray(clipped_raster[0], dims=['lat', 'lon'], coords={'lon': range(clipped_raster.shape[2]), 'lat':range(clipped_raster.shape[1])}) 

#         out_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI) + '\\' + region
#         if not os.path.exists(out_path):
#             os.makedirs(out_path)
#         out_file = out_path + '\\DDFs_' + str(hr) + '_' + region + '.tif'
#         with rasterio.open(out_file, 'w', **meta) as dst:
#             dst.write(data_array.values, 1)
        

In [906]:
# for region in regions:
#     files = glob.glob(r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI) + '\\' + region + '\\*.tif')
#     data_array = xr.concat([xr.open_rasterio(file) for file in files], dim='hrs')
#     data_array = data_array.rename({'x': 'lon', 'y': 'lat'})
#     data_array = data_array.assign_coords(lon=data_array.lon, lat=data_array.lat)
#     data_array.attrs['hrs'] = hrs
#     data_array = data_array.assign_coords(hrs=data_array.hrs)
#     data_array = data_array.rename('ddf')
#     data_array.to_netcdf(r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI) + '\\DDFs_' + region + '.nc')

In [907]:
# dataset = xr.open_dataset(r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI100\\Pt_ESI.nc')
# variable_names = list(dataset.variables)
# variable_names
# for var_name in variable_names:
#     var_values = dataset[var_name].values
#     print(f"Variable: {var_name}")
#     print(f"Values: {var_values}")
#     print()

In [908]:
# regions = []

# times = his_pdfs_df.index.values
# ntimes = len(times)

# for index, row in gdf_nz_bf.iterrows():
    
#     if index == 5:
        
#         region = row[1]
#         regions.append(region)

#         geometry = row.geometry
#         # # Perform the raster clipping
#         # clipped_raster, clipped_transform = mask(dataset=rasterio.open(P24_file), shapes=[geometry], crop=True)
#         # # Update the metadata with the clipped raster dimensions and transformation
#         # meta.update({'height': clipped_raster.shape[1], 'width': clipped_raster.shape[2], 'transform': clipped_transform})
#         # arr = clipped_raster[0]

#     # # for i in gdf_nz_bf.index:
#     # for i in [1]:

#     #     gdf = gdf_nz_bf.iloc[[i]]
#     #     region_name = gdf.iloc[0]['CZ']
#     #     geometry = gdf.geometry


#         data_array_list = []
#         for hr in hrs:
#             his_ddf_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI) + '\\Pt_' + str(hr) + '_NZ.tif'
#             with rasterio.open(his_ddf_file) as src:
#                 meta = src.meta.copy()
#             clipped_raster, clipped_transform = mask(dataset=rasterio.open(his_ddf_file), shapes=[geometry], crop=True)
#             # Update the metadata with the clipped raster dimensions and transformation
#             meta.update({'height': clipped_raster.shape[1], 'width': clipped_raster.shape[2], 'transform': clipped_transform})
#             data_array = xr.DataArray(clipped_raster[0], dims=['lat', 'lon'], coords={'lon': range(clipped_raster.shape[2]), 'lat':range(clipped_raster.shape[1])}) 
#             data_array_list.append(data_array)

#             # with rasterio.open('new.tif', 'w', **meta) as dst:
#             #     dst.write(data_array.values, 1)

#         data_arrays = xr.concat(data_array_list, dim='hrs')
#         hrs_values = hrs  # Adjust the range as needed
#         data_arrays = data_arrays.assign_coords(hrs=hrs_values)
#         data_arrays = data_arrays.rename('ddf')

#         # data_file = r'E:\Tuflow\Cal_rainfall_timeseries\output\baseline\DDFs' + '\\ARI' + str(ARI) + '\\Pt_' + region + '.nc'
#         # dataset = xr.Dataset({'data_array': data_arrays})
#         # dataset['hrs'] = data_arrays['hrs']
#         # dataset.to_netcdf(data_file, mode='w')
        
#         # for hr in hrs:
#         #     subset = data_arrays.sel(hrs=hr)
#         #     output_file = 'DDFs_' + region + '_' + str(hr) + '.tif'
#         #     subset = subset.rio.set_spatial_dims('lon', 'lat')
#         #     subset.rio.to_raster(output_file)



#         df_cf_region = df_cf.loc[region]

#         # for year, dgmt in df_dgmt[ssp].iteritems():
#         for dgmt in [4.64]:
#             year = 2100

#             # nrows, ncols = data_arrays.shape[1:]
#             # result_arr = np.zeros((nrows, ncols, ntimes))
#             fut_ddfs_list = []
#             for lat in data_arrays.coords['lat']:
#                 for lon in data_arrays.coords['lon']:
#                     data_value = data_arrays.sel(lat=lat, lon=lon)
#                     if data_value[0] != 0:
#                         his_ddfs = data_value.to_dataframe()
#                         his_ddfs = his_ddfs.rename(index=lambda x: x * 60, level='hrs')
#                         fut_ddfs = future_ddfs(dgmt, df_cf_region, his_ddfs)
#                         # fut_crrs = temporal_curve_24hr(fut_ddfs)
#                     else:
#                         his_ddfs = data_value.to_dataframe()
#                         fut_ddfs = his_ddfs.copy()
#                     fut_ddfs_list.append(fut_ddfs)
            
#             result = pd.concat(fut_ddfs_list)
#             B = xr.DataArray.from_series(result, coords=[data_arrays['lat'], data_arrays['lon']], name='ddf')
#             # B['lat'] = data_arrays['lat']
#             # B['lon'] = data_arrays['lon']

#     #         # save time series as tif
#     #         for ind_time in range(len(times)):

#     #             time = times[ind_time]
#     #             data = result_arr[:, :, ind_time]

#     #             out_path = r'E:\Tuflow\Cal_rainfall_timeseries\output\NZ\\'+ ssp + '_' + ens_ptl + '_' +  year + '_' + '\ARI' + str(ARI) + '\\Pt_region' 
#     #             if not os.path.exists(out_path):
#     #                 os.makedirs(out_path)
#     #             out_tif = out_path + '\\Pt_ARI' + str(ARI) + '_' + region + '_' + str(time) + '.tif'
#     #             with rasterio.open(out_tif, 'w', **meta) as dst:
#     #                 dst.write(data, 1)

# print('finished!')

In [909]:
fut_ddfs

NameError: name 'fut_ddfs' is not defined

In [ ]:
fut_ddfs_list

In [ ]:
his_ddfs, fut_ddfs

In [ ]:
data_arrays

In [ ]:
data_arrays.shape[1:]

In [ ]:
times